In [1]:
from Preprocessing import Preprocessing
from utils import Behaviour, Baseline
import Experiments
import matplotlib
import matplotlib.pyplot as plt
from multiprocessing import Pool
import math
matplotlib.use('Agg') # Prevent drawings in this notebook

In [2]:
bids_root = "../data/"
preprocessing = Preprocessing(bids_root, "cache")
# create cache files
preprocessing.prepare_channels()

In [3]:
combinations = [(i, behaviour) for i in range(65) for behaviour in Behaviour]
combinations.sort(key=lambda x: x[0])

process_count = 16
inputs = []
step_size = math.ceil(len(combinations)/process_count)
# split combinations to create inputs for processes
for i in range(process_count):
    inputs.append((combinations[step_size*i:step_size*(i+1)], preprocessing))

with Pool(process_count) as p:
    result = p.starmap(Experiments.run, inputs)

100%|██████████| 29/29 [10:45<00:00, 22.26s/it]

100%|██████████| 29/29 [10:51<00:00, 22.47s/it]


In [7]:
# join results from multiple processes
peak_to_troughs = {}
for behavior in Behaviour:
    peak_to_troughs[behavior] = {}
for i in range(len(result)):
    for behavior in result[i]:
        peak_to_troughs[behavior].update(result[i][behavior])

In [12]:
def plot_peak_to_trough_differences(behaviour, baseline, mode, upper=True):
    data = []
    for electrode_index, peak_to_trough_value in peak_to_troughs[behaviour].items():
        data.append((electrode_index, peak_to_trough_value[mode][baseline]))
    title = f"{'Upper' if upper else 'Lower'} Peak-to-Trough Differences for {behaviour.value} ({'Raw' if mode == 0 else 'ICA'}, {baseline.value})"
    if upper:
        sorted_values = sorted(data, key=lambda x:x[1])[-20:]
    else:
        sorted_values = sorted(data, key=lambda x:x[1])[:20]
    labels, values = zip(*sorted_values)
    plt.figure()
    plt.barh(range(len(values)), values, color='skyblue')
    plt.xlabel("Peak-to-Trough Difference")
    plt.ylabel("Electrode")
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.savefig(f"../Results/{title}.png")
    plt.close()    

In [13]:
# generate bar plots with peak-to-through differences
for behavior in peak_to_troughs:
    for baseline in Baseline:
        plot_peak_to_trough_differences(behavior, baseline, 0)
        plot_peak_to_trough_differences(behavior, baseline, 1)
        plot_peak_to_trough_differences(behavior, baseline, 0, False)
        plot_peak_to_trough_differences(behavior, baseline, 1, False)